# Graph Link Compression

This can only be done after the successful elimination of false linkages between contigs

Consider the example graph, with contig coverages as follows and
```python
contig_coverage = {
    'c2': 50,
    'c1': 150,
    'c3': 250,
    'c4': 50
}
```
link coverages as below
```python
link_coverage = {
    'c2+c1+': 50,
    'c1+c1+': 50,
    'c1+c3+': 50,
    'c3+c4+': 50,
    'c3+c3+': 150,
    'c4+c3+': 50
}
```

This is taken from the genome = `c2-c1-c1-c3-c3-c4-c3-c3-c3`

The compressed expression is = `c2-c1*x-c3*y-c4-c3*z`
In reality derivation of `y` and `z` is nearly impossible, rather an expression generalizing all possible arrangements is possible. For this the coverage information can be used.

In [14]:
# all the imports
# python imports
import copy

# initialize with sample data
# contig_coverage = {
#     'c1': 50,
#     'c2': 150,
#     'c3': 250,
#     'c4': 50,
#     'c5': 50
# }

# link_coverage = {
#     'c1+c2+': 50,
#     'c2+c2+': 50,
#     'c2+c3+': 50,
#     'c3+c3+': 50,
#     'c3+c2+': 150,
#     'c2+c4+': 50,
#     'c4+c2+': 50,
#     'c2+c5+': 50
# }

contig_coverage = { 'contig_3-': 1, 'contig_2+': 3, 'contig_1+': 1, 'contig_4+': 10}

link_coverage = {'contig_4+contig_2+': 11, 'contig_1+contig_4+': 3, 'contig_2+contig_4+': 49, 'contig_4+contig_3-': 8}



read_coverage = 50

links = list(link_coverage.keys())
contigs = list(contig_coverage.keys())

print (links)
print ()
print (contigs)

['contig_4+contig_2+', 'contig_1+contig_4+', 'contig_2+contig_4+', 'contig_4+contig_3-']

['contig_3-', 'contig_2+', 'contig_1+', 'contig_4+']


In [16]:
class Graph:
    
    def __init__(self, links, contig_coverage_dict, link_coverage_dict):
        self._links = links
        self._graph = None
        self._graph_p = None
        self.contig_coverage_dict = contig_coverage_dict
        self.link_coverage_dict = link_coverage_dict

    def _name_decomposer(self, link_name):
        decomposed = []
        temp = ""

        for c in link_name:
            if c == "+" or c == "-":
                contig_name = temp
                temp = ""
                decomposed.extend([contig_name, c])
            else: temp += c
        return decomposed

    def _build_graph(self):
        graph = {}
        graph_p = {}

        for link in self._links:
            l_list = self._name_decomposer(link)
            l_1 = "".join(l_list[0:2])
            l_2 = "".join(l_list[2:4])

            if l_1 not in graph:
                graph[l_1] = set()
            if l_2 not in graph:
                graph[l_2] = set()

            if l_2 not in graph_p:
                graph_p[l_2] = set()
            if l_1 not in graph_p:
                graph_p[l_1] = set()

            # avoid cycles
            if l_1 == l_2: continue

            graph[l_1].add(l_2)
            graph_p[l_2].add(l_1)
 
        print("INFO:: graph", graph)
        print("INFO:: graph_p", graph_p)
        
        self._graph, self._graph_p = graph, graph_p
    
    def _get_cycle_dfs(self, start, end):
        graph = self._graph
        fringe = [(start, [])]
        while fringe:
            state, path = fringe.pop()
            if path and state == end:
                yield path
                continue
            for next_state in graph[state]:
                if next_state in path:
                    continue
                fringe.append((next_state, path+[next_state]))
            
    # obtain vertices that are worth looking for cycles
    def _obtain_critical_vertices(self):
        none_set = []
        one_set = []
        many_set = []
        
        one_link_set = []
        
        for key, val in self.contig_coverage_dict.items():
            if val == 1:
                none_set.append(key)
            elif val == 2:
                one_set.append(key)
            else:
                many_set.append(key)
                
        # TODO do something with such links
        
        return none_set, one_set, many_set
        
    def _intersection(self, lst1, lst2): 
        lst3 = [value for value in lst1 if value in lst2] 
        return lst3 

    def _trace(self):
        none_set, one_set, many_set = self._obtain_critical_vertices()
        discovered_ones = set()
        unique_cycles = []
        recurrent_cycles = []
        
        for vertex in one_set:
            cycles = [[vertex] + x for x in self._get_cycle_dfs(vertex, vertex)]
            unique_cycles.extend(cycles)
            for cycle in cycles:
                discovered_ones.update(cycle)
            
        
        for vertex in many_set:
            recurrent_cycles.extend([[vertex] + x for x in self._get_cycle_dfs(vertex, vertex)])
            
        # TODO decide what to do if there are multiplye cycles for unique cycles
        
        # recurrent cycles must not contain elements from none_set if discovered
        temp = recurrent_cycles
        recurrent_cycles = []
        
        while len(temp) > 0:
            cycle = temp.pop()
            
            if len(self._intersection(self._intersection(cycle, none_set), list(discovered_ones))) > 0:
                continue
            else:
                discovered_ones.update(cycle)
                recurrent_cycles.append(cycle)
            
            
        
        for vertex in none_set:
            if not vertex in discovered_ones:
                if self._graph_p[vertex]:
                    unique_cycles.append([list(self._graph_p[vertex])[0], vertex])
                else:
                    unique_cycles.append([vertex, list(self._graph[vertex])[0]])

        
        print(self._obtain_critical_vertices())
        
        print("unique cycles", unique_cycles)
        print("recurrent cycles", recurrent_cycles)
        
        return unique_cycles, recurrent_cycles
        
    
    def run_resolution(self):
        self._build_graph()
        
#         if self._ensure_euler_path():
        return self._trace()
#         else:
#             raise Exception('Unable to secure an euler path')
        
        
# eg = EulerGraph(['c1+c2+', 'c2+c2+', 'c2+c3+', 'c3+c3+', 'c3+c2+', 'c2+c4+', 'c4+c2+', 'c2+c5+']) 
# eg = EulerGraph(['c1+c5+', 'c1+c2+', 'c2+c3+', 'c3+c4+', 'c4+c1+', 'c4+c6+',  'c5+c3+',  'c4+c3+', 'c6+c3+']) 
# eg = Graph(['c1+c5+', 'c1+c2+', 'c2+c3+', 'c3+c4+', 'c4+c3+', 'c4+c6+', 'c4+c1+', 'c6+c3+'],
#            {'c1+':2, 'c2+':1,'c6+': 1, 'c3+': 4, 'c4+': 4, 'c5+': 1},
#            {}
#           ) 

eg = Graph(link_coverage.keys(),
           contig_coverage,
           {}
          ) 

trails = eg.run_resolution()

for x in trails:
    print (x)
# euler_trace(graph, graph_p)

INFO:: graph {'contig_4+': {'contig_2+', 'contig_3-'}, 'contig_2+': {'contig_4+'}, 'contig_1+': {'contig_4+'}, 'contig_3-': set()}
INFO:: graph_p {'contig_2+': {'contig_4+'}, 'contig_4+': {'contig_2+', 'contig_1+'}, 'contig_1+': set(), 'contig_3-': {'contig_4+'}}
(['contig_3-', 'contig_1+'], [], ['contig_2+', 'contig_4+'])
unique cycles [['contig_4+', 'contig_3-'], ['contig_1+', 'contig_4+']]
recurrent cycles [['contig_4+', 'contig_2+', 'contig_4+'], ['contig_2+', 'contig_4+', 'contig_2+']]
[['contig_4+', 'contig_3-'], ['contig_1+', 'contig_4+']]
[['contig_4+', 'contig_2+', 'contig_4+'], ['contig_2+', 'contig_4+', 'contig_2+']]


In [210]:
# special comments

if a node has no children or parent => coverage must be 1

[['c1+', 'c2+', 'c3+', 'c4+', 'c1+']]